In [1]:
# Import libraries
import matplotlib.pyplot as plt
import math
import nltk
import numpy as np
import pandas as pd
import re
import seaborn as sns
import string

from collections import Counter
from numpy import sqrt 
from tqdm import tqdm

from sklearn import metrics
from sklearn.metrics import mean_squared_error

from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors

from scipy.sparse import csr_matrix

In [2]:
from google.colab import drive

Define Data

In [3]:
# Connect to GDrive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
dataset = pd.read_csv('/content/drive/MyDrive/KULIAH/TA-2/SIDANG/TASI-107/DATA/Data_Preprocessing.csv')
dataset.sample(3)

,reviewerId,placeId,destination_name,description,category_name,text,publishedAtDate,user_rating,overview
10407,2803,62,Hatulian Beach,Pantai Hatulian adalah wisata air dan alam yan...,wisata bahari,(Translated by Google) Cool\n\n(Original)\nSejuk,2020-08-06T09:39:02.438Z,5,wisata bahari pantai hatulian wisata air alam ...
5523,4824,57,Pantai Lumban Bul-bul Balige,Tempat wisata di Toba Samosir yang kedua adala...,wisata bahari,.,2018-04-11T12:45:10.312Z,5,wisata bahari wisata toba samosir wisata panta...
5458,4768,57,Pantai Lumban Bul-bul Balige,Tempat wisata di Toba Samosir yang kedua adala...,wisata bahari,.,2019-01-07T09:55:59.517Z,5,wisata bahari wisata toba samosir wisata panta...


# CF

In [5]:
CF=dataset[['placeId','reviewerId', 'user_rating']]
CF.head()

,placeId,reviewerId,user_rating
0,1,0,5
1,1,1,5
2,1,2,5
3,1,3,5
4,2,4,5


In [6]:
#90% baris data dipilih secara random sebagai dara train
CF_train = CF.sample(frac = 0.9, random_state = 1)
CF_train = CF_train.reset_index(drop=True)
CF_test = CF.drop(CF_train.index)
CF_test = CF_test.reset_index(drop=True)

In [7]:
x_train = CF_train[['reviewerId','placeId']]
y_train = CF_train[['user_rating']]
x_test = CF_test[['reviewerId','placeId']]
y_test = CF_test[['user_rating']]

In [8]:
x_train

,reviewerId,placeId
0,6595,58
1,4917,57
2,2959,53
3,487,32
4,4272,57
...,...,...
9579,6126,58
9580,7284,59
9581,7343,59
9582,3244,57


In [9]:
x_test

,reviewerId,placeId
0,7990,60
1,7991,60
2,749,60
3,7992,60
4,7993,60
...,...,...
1060,8747,65
1061,2679,65
1062,8748,65
1063,8749,65


In [10]:
# the number of users
n_users = CF_train.reviewerId.unique().shape[0]

# the number of items in the dataset
n_items = CF_train.placeId.unique().shape[0]

#max reviewerId
n_users = CF_train['reviewerId'].max()

#max placeId
n_items = CF_train['placeId'].max()

#initialize empty array
A = np.zeros((n_users,n_items))


In [11]:
# mengubah kumpulan data menjadi kumpulan data biner

for i in range (len(A)):
  for j in range(len(A[0])):
    if A[i][j]>=3:
      A[i][j]=1
    else:
      A[i][j] = 0



In [12]:
matrix = CF.pivot(index='reviewerId',columns='placeId',values='user_rating').fillna(0)

display(matrix.head())

placeId,1,2,3,4,5,6,7,8,9,10,...,56,57,58,59,60,61,62,63,64,65
reviewerId,,,,,,,,,,,,,,,,,,,,,
0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
3,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
4,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
print("Original rating materix :  ",A)

Original rating materix :   [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [14]:
#konversi matrix untuk menghemat memori
sparse_matrix = csr_matrix(A)

###Mengisi data sparse dengan rata-rata rating


In [15]:
import statistics  

In [16]:
def calculate_rating(list_item_id, CF):
  list_rating_tmp = []

  for i in list_item_id:
    avg_rating = CF[CF["reviewerId"] == i][["user_rating"]].mean()
    list_rating_tmp.append(float(avg_rating))

    return round(statistics.mean(list_rating_tmp), 2)
  
def predict_nearest_item(reviewerId, list_item, sparse_matrix, model_knn, k):
  list_index = []

  for i in list_item:
    distances, indices = model_knn.kneighbors(sparse_matrix[i],
                                               n_neighbors=k)
    indices = indices.flatten()
    indices = indices[1:]
    list_index.extend(indices)

  return list_index

def knn_predict(df_test, sparse_matrix, model_knn, k):
  list_rating_tmp = []
  for i in df_test['reviewerId']:
    filter_user = df_test[df_test["reviewerId"] == i].placeId
    filter_user = filter_user.tolist()
    list_item = predict_nearest_item(i, filter_user, sparse_matrix, model_knn, k)
    list_rating_tmp.append(calculate_rating(list_item, CF))
  return np.array(list_rating_tmp)

In [17]:
def eksperimen_hiperparameter_knn(matriks_rating, x_uji, y_uji, k):
  knn = NearestNeighbors(n_neighbors=k, algorithm='brute', p = 2)
  knn.fit(matriks_rating)
  pred2 = knn_predict(x_uji, matriks_rating, knn, k)
  rmse = np.sqrt(np.mean((pred2 - y_uji.to_numpy())**2))
  return rmse

In [18]:
print("matriks_rating")

matriks_rating


In [19]:
list_k = [3, 5, 10, 20, 25, 30]
list_rmse_nonsparse = []
for i in list_k:
  rmse_val = eksperimen_hiperparameter_knn(sparse_matrix, x_test, y_test, i)
  list_rmse_nonsparse.append(rmse_val)
  print("nilai k =", i, "RMSE = %.4f" %rmse_val)

nilai k = 3 RMSE = 1.0963
nilai k = 5 RMSE = 1.0760
nilai k = 10 RMSE = 1.0760
nilai k = 20 RMSE = 1.0963
nilai k = 25 RMSE = 1.0963
nilai k = 30 RMSE = 1.0963
